In [1]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import time

In [2]:
from sklearn.metrics import *
from sklearn.model_selection import *

In [3]:
df = pd.read_csv('train_nov28_task1.csv') #讀入訓練資料
df.head()

,feature0,feature1,feature2,feature3,feature4,class
0,4.115910,8.678126,16.010297,5.731837,18.907271,A
1,1.993343,8.920002,15.332958,10.389251,0.023662,A
2,0.095994,8.847990,7.292827,4.059212,7.914817,A
3,3.808525,8.726179,13.842177,9.997444,0.178466,A
4,4.554317,8.850917,15.659514,8.053744,0.341788,A


In [4]:
y = df['class'] #訓練資料分類結果
X = df.drop('class', axis = 1) #X中留下features

In [5]:
#訓練資料分割成0.85訓練、0.15測試
X1, X2, y1, y2 = train_test_split(X, y, random_state=0, train_size=0.85, test_size=0.15)

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
#建立參數的各自區間
n_estimators = [int(x) for x in np.linspace(start=1000, stop=10000, num=10)]
max_features = ['auto', 'sqrt', 'log2']

max_depth = [None]
min_samples_split = [2]
min_samples_leaf = [1]
bootstrap = [False]

random_grid = {'n_estimators': n_estimators, 'max_features': max_features,
               'max_depth': max_depth, 'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}
random_grid


{'n_estimators': [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000],
 'max_features': ['auto', 'sqrt', 'log2'],
 'max_depth': [None],
 'min_samples_split': [2],
 'min_samples_leaf': [1],
 'bootstrap': [False]}

In [19]:
#用RandomizedSearchCV來找對訓練資料而言最好的參數
forest2 = RandomForestClassifier(random_state=42)
rf_random = RandomizedSearchCV(estimator = forest2, param_distributions=random_grid,
                              n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

rf_random.fit(X1,y1)
rf_random.best_params_

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 30 is smaller than n_iter=100. Running 30 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


{'n_estimators': 6000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': None,
 'bootstrap': False}

In [20]:
#用RandomizedSearchCV找到的最好的參數做訓練
from sklearn.ensemble import RandomForestClassifier

RFC_pred = RandomForestClassifier(n_estimators = 6000,
 min_samples_split = 2,
 min_samples_leaf = 1,
 max_features = 'sqrt',
 max_depth = None,
 bootstrap = False).fit(X1, y1).predict(X2) 
print(accuracy_score(y2, RFC_pred))

0.9425925925925925


In [21]:
df_test = pd.read_csv('test_nov28_task1_only_features.csv') #讀入測試資料

In [22]:
#用RandomizedSearchCV找到的最好的參數訓練並預測
RFC_pred_fianl = RandomForestClassifier(n_estimators = 6000,
 min_samples_split = 2,
 min_samples_leaf = 1,
 max_features = 'sqrt',
 max_depth = None,
 bootstrap = False).fit(X, y).predict(df_test) 

In [23]:
import csv

In [24]:
#寫入csv
with open('output_RFC_2.csv', 'w', newline='') as csvfile:
  
    writer = csv.writer(csvfile)

    writer.writerow(['Id', 'Category'])

    for i in range(len(RFC_pred_fianl)):
        writer.writerow([i + 1, RFC_pred_fianl[i]])